## Bölüm 0: Kurulum ve Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

print(f"PyTorch Version: {torch.__version__}")
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current Device: {device}")
print(f"CPU Count: {torch.get_num_threads()}")

Soru 1: Neural Network yapmak için hangi kütüphaneler lazım?

Cevap:
- torch: PyTorch'un ana modülü
- torch.nn: Layer, activation fonksiyonları vs.
- torch.optim: Optimizer'lar
- numpy: Sayısal işlemler
- matplotlib: Grafik çizimi

Soru 2: GPU'da mı CPU'da mı çalıştırırız?

Cevap:
- İlk torch.cuda.is_available() ile GPU var mı kontrol et
- device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
- Model ve veriyi .to(device) ile gönder
- GPU'da çalışırsa çok daha hızlı

## Bölüm 1: PyTorch Temelleri ve Basit Model

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

X = torch.randn(100, 5, device=device)
y = torch.randn(100, 1, device=device)

print(f"Input shape: {X.shape}")
print(f"Target shape: {y.shape}")

Soru 1: Rastgele veri nasıl oluşturuyoruz?

Cevap:
- torch.randn() ile random sayılar
- X = torch.randn(100, 5) → 100 örnek, 5 feature
- y = torch.randn(100, 1) → target değişken
- .to(device) ile GPU'ya gönder

Soru 2: Model nasıl yazarız?

Cevap:
- nn.Module extend et
- __init__ de layer'ları tanımla
- forward() de veri flow'unu yaz
- x → layer1 → relu → layer2 → output

Soru 3: Eğitim döngüsü nasıl?

Cevap:
1. Forward: tahmin = model(X)
2. Loss: hata = loss_fn(tahmin, y)
3. Gradients temizle: optimizer.zero_grad()
4. Backward: hata.backward()
5. Güncelle: optimizer.step()

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size=5, hidden_size=10, output_size=1):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = SimpleNN(input_size=5, hidden_size=10, output_size=1).to(device)
print("Model:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")

In [ ]:
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 50
losses = []

print("Training the model...")
for epoch in range(num_epochs):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}")

plt.figure(figsize=(10, 5))
plt.plot(losses, linewidth=2)
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.title("Training Loss Over Epochs")
plt.grid(True, alpha=0.3)
plt.show()

## Bölüm 2: Yapay Sinir Ağı - XOR Problemi

Soru 1: XOR problemi nedir?

Cevap:
- XOR lineer değil, tek layer'la çözemezsin
- Hidden layer eklersin non-linear hale gelir
- Input(2) → Hidden(4) → Output(1)
- ReLU ve Sigmoid kullanırsın

Soru 2: Matematiksel işlemler nasıl?

Cevap:
- Her layer: z = W×x + b
- Activation: ReLU(z)
- Tersine: chain rule ile gradient
- Her layer'dan gradient geri gidiyor

Soru 3: Gradient nedir?

Cevap:
- Gradient = loss'un ağırlığa göre değişim
- Nereye gidersek loss azalır bunu gösteriyor
- w = w - learning_rate × gradient
- Bunu yapmasak ağ öğrenemezdi

Soru 4: Overfitting, epoch, veri split?

Cevap:
- Overfitting: ağ eğitim verisine çok alışıyor
- Training loss azalır, test loss yükselir
- Epoch: tüm veriyi bir kez geçirmek
- Veriyi train/test'e böl, overfitting kontrol et

In [ ]:
X_xor = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32, device=device)
y_xor = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32, device=device)

print("XOR Dataset:")
print(f"Inputs:\n{X_xor}")
print(f"Targets:\n{y_xor}")

In [ ]:
class XOR_NN(nn.Module):
    def __init__(self):
        super(XOR_NN, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

xor_model = XOR_NN().to(device)
xor_optimizer = optim.Adam(xor_model.parameters(), lr=0.01)
xor_loss_fn = nn.BCELoss()

xor_losses = []
xor_epochs = 1000

print("Training XOR model...")
for epoch in range(xor_epochs):
    y_pred = xor_model(X_xor)
    loss = xor_loss_fn(y_pred, y_xor)
    xor_optimizer.zero_grad()
    loss.backward()
    xor_optimizer.step()
    
    xor_losses.append(loss.item())
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch [{epoch+1}/{xor_epochs}], Loss: {loss.item():.6f}")

print("\nXOR Model Predictions:")
with torch.no_grad():
    predictions = xor_model(X_xor)
    for i in range(len(X_xor)):
        print(f"Input: {X_xor[i].cpu().numpy()} → Predicted: {predictions[i].item():.4f}, Target: {y_xor[i].item()}")

## Bölüm 3: Matematiksel Tanımlar

Soru 1: Activation function'lar?

Cevap:
- ReLU: En çok kullanılan, hızlı, hidden layer'lar için
- Sigmoid: 0-1 arasında, binary classification output
- Tanh: -1 ile 1 arasında, sigmoid'den iyi
- Leaky ReLU: ReLU'nun ölü nöron problemi çözüyor
- Softmax: Multi-class classification

Soru 2: Loss function'lar?

Cevap:
- MSE: Regression, tahmin ve gerçek farkının karesi
- Cross Entropy: Classification, ne kadar yanlış
- Binary Cross Entropy: 2 sınıflı problemler
- L1 Loss: Regression, aykırı değerlere dayanıklı

Soru 3: Gradient descent nasıl?

Cevap:
- Gradient hesapla
- w = w - learning_rate × gradient
- Tekrar tekrar
- Loss azalana kadar

Soru 4: NN'nin matematiksel temeli?

Cevap:
- Linear Algebra: Matrisler, vektörler
- Calculus: Türev, chain rule
- Probability: Softmax, olasılıklar
- Optimization: Gradient descent, Adam

## Bölüm 4: Veri Akışı ve Yaklaşımlar

Soru 1: Bir problem nasıl çözeriz NN ile?

Cevap:
1. Problemi anla
2. Veriyi hazırla
3. Model tasarla
4. Activation ve loss seç
5. Optimizer aç
6. Eğit
7. Sonucu kontrol et

Soru 2: Veri ağda nasıl akar?

Cevap:
- Input layer
- Hidden layers: Veri dönüştürülüyor
- Output layer: Sonuç
- Tersine: Loss'tan başlayıp gradient gidiyor

## Bölüm 5: Değerlendirme ve Görevler

Soru: PyTorch ve NN projesi yap

Cevap:

Fashion MNIST ile sınıflandırma ağı yaptık.

Model yapısı:
- Input: 784 nöron (28×28 görüntü)
- Hidden 1: 256 nöron + ReLU + Dropout
- Hidden 2: 128 nöron + ReLU + Dropout  
- Hidden 3: 64 nöron + ReLU + Dropout
- Output: 10 nöron (10 sınıf)

Eğitim:
- Adam optimizer
- CrossEntropyLoss
- 15 epoch
- Batch size 64

Sonuçlar:
- Loss grafikleri
- Accuracy grafikleri
- Confusion matrix
- Detaylı rapor

Proje tamamlandı.